# Máquinas de Vectores de Soporte (SVM)
## Presentación - Encuentro 4

---

## 1. ¿Qué es SVM?

### Concepto Simple
Las **Máquinas de Vectores de Soporte (SVM)** son algoritmos que encuentran la **mejor manera de separar** diferentes grupos de datos.

### Analogía Visual
Imagine que tiene **perros** y **gatos** mezclados en un patio. El SVM encuentra la **línea más amplia** entre ellos para separarlos claramente.

---

## 2. Los 4 Conceptos Fundamentales

### 2.1 Hiperplano
- **Qué es**: La línea (o plano) que separa las clases
- **Objetivo**: Buscar la línea que esté lo más lejos posible de ambos grupos
- **Visualización**:
  - 2D: una línea recta
  - 3D: un plano
  - nD: un hiperplano (generalización)

    ![](svm-clasificador-c.png)

    [Imagen extraida de Luis G. Serrano - Machine Learning]()

### 2.2 Vectores de Soporte
- **Qué son**: Los puntos de datos **más cercanos** a la línea de separación
- **Importancia**: Solo necesitas estos puntos para clasificar nuevos datos
- **Ventaja**: Muy eficiente en memoria

  **Ejemplo**: 1000 muestras $\to$ 50 vectores (puntos) de soporte

### 2.3 Margen
- **Qué es**: El espacio libre entre la línea de separación y los puntos más cercanos
- **Objetivo**: Maximizar este espacio para crear una separación más robusta
- **Tipos**:
  - **Margen duro**: No permite errores
  - **Margen suave**: Permite algunos errores (más flexible)

    ![](svm-clasificador-a.png)

    [Imagen extraida de Luis G. Serrano - Machine Learning]()

### 2.4 Kernel
- **Qué es**: Una función que permite separaciones **no lineales** (curvas)
- **Tipos principales**:
  - **Lineal**: Separaciones rectas (más rápido)
  - **RBF**: Separaciones curvas complejas (más flexible)
  - **Polinomial**: Separaciones con curvas polinómicas
- **Cuándo usar**: RBF para casos generales

---

## 3. Parámetros Principales

### Parámetro C (Regularización)
| Valor             | Significado                            | Uso Recomendado               |
|-------------------|----------------------------------------|-------------------------------|
| C grande (>100)   | Línea rígida, pocos errores permitidos | Cuando quieres alta precisión |
| C pequeño (<1)    | Línea flexible, más errores permitidos | Cuando los datos tienen ruido |
| **Valor inicial** | C=1                                    | Punto de partida estándar     |

### Parámetro gamma (Solo para kernel RBF)
| Valor                | Significado          | Uso Recomendado         |
|----------------------|----------------------|-------------------------|
| gamma grande (>1)    | Curvas muy complejas | Riesgo de sobreajuste   |
| gamma pequeño (<0.1) | Curvas suaves        | Mejor generalización    |
| **Valor inicial**    | gamma=0.1            | Valor seguro y estándar |

### Grid Search
- **Qué es**: Prueba automáticamente diferentes combinaciones de parámetros
- **Ventaja**: Encuentra la mejor combinación automáticamente
- **Resultado**: Te entrega el modelo ya optimizado

---

## 4. Flujo de Trabajo con SVM

### Paso a Paso

1. **Preparar datos**
   - **CRÍTICO**: Normalizar con StandardScaler
   - SVM es muy sensible a la escala de los datos

2. **Dividir datos**
   - 80% para entrenamiento
   - 20% para prueba

3. **Grid Search**
   - Prueba diferentes combinaciones automáticamente
   - Ya entrena el modelo con los mejores parámetros

4. **Evaluar y visualizar**
   - Ver las fronteras de decisión
   - Calcular métricas de rendimiento

### Ejemplo: Comparación de SVM con GridSearchCV

In [ ]:
# SVM (lineal, RBF y polinomial), GridSearchCV, 5-Fold y PCA para visualización

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, classification_report
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# 1. Generar datos sintéticos (puedes cambiar n_features > 2 para probar PCA)
X, y = make_blobs(n_samples=400, centers=2, n_features=5, cluster_std=4, random_state=42)

# 2. Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 3. Definir la grilla de hiperparámetros
param_grid = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10]},
    {'kernel': ['rbf'], 'C': [0.1, 1, 10], 'gamma': ['scale', 0.1, 1]},
    {'kernel': ['poly'], 'C': [0.1, 1, 10], 'degree': [2, 3, 4], 'gamma': ['scale', 0.1, 1]}
]

# 4. Crear el modelo base y el GridSearchCV con validación cruzada (5 folds)
svm = SVC()
grid = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

# 5. Mostrar los mejores parámetros
print("Mejor combinación encontrada por GridSearchCV:")
print(grid.best_params_)
print(f"Mejor puntuación media de validación: {grid.best_score_:.3f}")

# 6. Evaluar el mejor modelo
best_svm = grid.best_estimator_
y_pred = best_svm.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"\nPrecisión en el conjunto de prueba: {acc:.3f}")
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# 7. Matriz de confusión
ConfusionMatrixDisplay.from_estimator(best_svm, X_test, y_test, cmap='Blues')
plt.title("Matriz de confusión del mejor SVM")
plt.show()

# 8. Reducción a 2D con PCA (solo para visualización)
if X.shape[1] > 2:
    print(f"\nAplicando PCA: reducción de {X.shape[1]} → 2 dimensiones para visualización.")
    pca = PCA(n_components=2)
    X_vis = pca.fit_transform(X)
else:
    X_vis = X  # si ya tiene 2 dimensiones, no aplicamos PCA

# 9. Visualización de fronteras de decisión
# Usamos PCA solo para graficar, no para entrenar el modelo (por claridad)
# En escenarios reales, puedes aplicar PCA antes del entrenamiento si lo deseas.

plt.figure(figsize=(8,6))

# Crear una malla 2D
x_min, x_max = X_vis[:, 0].min() - 1, X_vis[:, 0].max() + 1
y_min, y_max = X_vis[:, 1].min() - 1, X_vis[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                     np.linspace(y_min, y_max, 300))

# Si los datos fueron reducidos con PCA, proyectamos la malla al espacio original para predecir
if X.shape[1] > 2:
    # Invertimos la proyección para estimar el espacio original
    grid_points = np.c_[xx.ravel(), yy.ravel()]
    grid_points_original = pca.inverse_transform(grid_points)
    Z = best_svm.predict(grid_points_original)
else:
    Z = best_svm.predict(np.c_[xx.ravel(), yy.ravel()])

Z = Z.reshape(xx.shape)

# Dibujar fronteras
plt.contourf(xx, yy, Z, alpha=0.3, cmap='Accent')
plt.scatter(X_vis[:, 0], X_vis[:, 1], c=y, cmap='Accent', edgecolors='k')
plt.title(f"Fronteras de decisión del mejor SVM ({grid.best_params_['kernel']})")
plt.xlabel("Componente 1 (PCA)" if X.shape[1] > 2 else "Característica 1")
plt.ylabel("Componente 2 (PCA)" if X.shape[1] > 2 else "Característica 2")
plt.show()


## 5. ¿Cuándo Usar SVM?

### Ventajas
- **Eficiente en memoria**: Solo almacena vectores de soporte
- **Bueno con alta dimensionalidad**: Funciona bien con muchas características
- **Flexible**: Diferentes kernels para diferentes tipos de separación
- **Robusto**: Resiste datos atípicos

### Desventajas
- **Lento con datasets muy grandes** (>10,000 muestras)
- **Requiere normalización** (no opcional)
- **Parámetros sensibles** (requieren ajuste)

### Cuándo Usar
- Dataset pequeño a mediano (<10,000 muestras)
- Necesitas separaciones no lineales
- Trabajas con alta dimensionalidad
- Necesitas un modelo robusto

---

## 6. Métricas de Evaluación

### Matriz de Confusión

| Real \ Predicción | Positiva        | Negativa        |
|-------------------|-----------------|-----------------|
| **Positiva**      | **VP** o **TP** | FN              |
| **Negativa**      | FP              | **VN** o **TN** |

#### Ejemplo:

| Real \ Predicho | Perro       | Gato        | Pájaro      |
|-----------------|-------------|-------------|-------------|
| **Perro**       | **70 (TP)** | 5 (FN)      | 2 (FN)      |
| **Gato**        | 10 (FP)     | **85 (TP)** | 5           |
| **Pájaro**      | 0 (FP)      | 3           | **95 (TP)** |


### Métricas Principales

**Fórmulas básicas:**
- **Accuracy**: (TP + TN) / Total
- **Precision**: TP / (TP + FP)  
- **Recall**: TP / (TP + FN)
- **F1-Score**: 2 × (Precision × Recall) / (Precision + Recall)

| Métrica       | Interpretación                                |
|---------------|-----------------------------------------------|
| **Accuracy**  | Porcentaje de clasificaciones correctas      |
| **Precision** | De los predichos positivos, ¿cuántos son realmente positivos? |
| **Recall**    | De los realmente positivos, ¿cuántos detectamos? |
| **F1-Score**  | Balancea precisión y recall (media armónica)  |

### ¿Qué Métrica Usar?

| Escenario                    | Métrica Principal | Razón                           |
|------------------------------|-------------------|---------------------------------|
| Clases balanceadas           | **Accuracy**      | Representa bien el rendimiento  |
| Clase minoritaria importante | **F1-Score**      | Balancea ambas métricas         |
| Evitar falsas alarmas        | **Precision**     | Ej: spam vs no-spam             |
| Detectar todos los casos     | **Recall**        | Ej: detección de fallas, cáncer |
| Clases desbalanceadas        | **Macro F1**      | No se sesga por mayoría         |

---

## 7. Mejores Prácticas

### Hacer SIEMPRE
1. **Normalizar datos** (crítico)
2. Usar **Grid Search** para encontrar mejores parámetros
3. **Validación cruzada** para evaluar robustez
4. Comparar **train vs test accuracy** para detectar sobreajuste
5. Empezar con valores conservadores de C y gamma

### Errores Comunes
1. **No normalizar** datos de prueba correctamente
2. **Usar datos de prueba durante entrenamiento**
3. **Ignorar sobreajuste** (train accuracy alta, test baja)
4. **Valores extremos** de C y gamma sin validar
5. **No visualizar** las fronteras de decisión

### Interpretación de Resultados
- **Pocos vectores de soporte**: Modelo simple y generalizable
- **Muchos vectores de soporte**: Modelo complejo (posible sobreajuste)
- **Train accuracy alta, test baja**: Sobreajuste (reducir C o gamma)
- **Baja accuracy en ambos**: Bajo rendimiento (cambiar kernel o parámetros)

---

## 8. El Camino de los Datos: Entrenamiento → Predicción

  ![](ml-svm-blocos_diagram_01.png)

---

## 9. Entrenamiento del modelo

  ![](ml-svm-blocos_diagram_02.png)

---

## 10. Puntos Clave para Recordar

### Los 3 Conceptos Esenciales
1. **Hiperplano**: La línea que separa clases
2. **Vectores de Soporte**: Puntos clave que definen la separación
3. **Margen**: Espacio que queremos maximizar

### Los 2 Parámetros Críticos
1. **C**: Controla cuánto errores permitir (C grande = menos errores)
2. **gamma**: Controla complejidad de curvas (gamma pequeño = más simple)

### El Flujo Clave
1. Normalizar (SIEMPRE)
2. Grid Search (optimización automática)
3. Visualizar (entender el modelo)
4. Evaluar (métricas adecuadas)

---

## 11. Próximos Pasos

### Para Practicar
1. Ejecutar el notebook de práctica
2. Probar diferentes kernels (lineal, RBF)
3. Cambiar parámetros C y gamma
4. Aplicar a dataset real (ej: Iris)
5. Experimentar con datos multiclase

### Para Profundizar
- Explorar SVM multiclase
- Implementar en casos industriales reales
- Comparar con otros algoritmos (Random Forest, Redes Neuronales)
- Estudiar teoría matemática de SVM

---

## 12. Recursos y Referencias

### Documentación
- Grokking Machine Learning, Luis G. Serrano, 2021
- [Scikit-learn: SVM](https://scikit-learn.org/stable/modules/svm.html)
- [Grid Search Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

### Para Aprender Más
- Libro: "Pattern Recognition and Machine Learning" - Christopher Bishop
- Curso: Machine Learning por Andrew Ng (Coursera)

---

## ¿Preguntas?

*Tiempo restante para preguntas y discusión*

---

## Apéndice: Comparación Rápida con Otros Algoritmos

| Algoritmo           | Interpretabilidad | Escalabilidad | Datos No Lineales | Velocidad |
|---------------------|-------------------|---------------|-------------------|-----------|
| **SVM**             | Media             | Baja          | Excelente         | Media     |
| Random Forest       | Alta              | Alta          | Buena             | Alta      |
| Redes Neuronales    | Baja              | Media         | Excelente         | Media     |
| Regresión Logística | Alta              | Alta          | Limitada          | Alta      |

